In [1]:
import pandas as pd
from random import randint
from random import choice
from random import seed
import time
seed(1)
from collections import defaultdict
datos_corredores = pd.read_csv("corredores.csv", sep=";", encoding="ascii" )
datos_carreras = pd.read_csv("lugares.csv", sep=";", encoding="ascii" )

In [2]:
X = ["juvenil", "sub_23", "elite", "masterA", "masterB", "masterC", "masterD", "masterE"]
# T  = sabados del año + domingos del año
T = list(range(6,366, 7)) + list(range(7,366, 7)) 
###
R = ['IV', 'RM', 'V', 'VI', 'VII', 'VIII', 'XIV']
I = datos_corredores.nombre.tolist()
P = datos_carreras["Carrera"].tolist()
for i in range(len(P)):
    P[i] = P[i] + "-" + str(i)


K_r = {}
for r in R:
    K_r[r] = randint(40, 100)


Q = ["c1","c2","c3","c4","c5","c6","c7","c8","c9","c10"]
A = 5000000
B = 4
E = list(range(17, 55))
L = []
L_r = defaultdict(list)
for i in R:
    for x in range(randint(1, 4)):
        L.extend([i+"/L"+ str(x)])
        L_r[i].append(i+"/L"+ str(x))


In [3]:

C_i = defaultdict(str)
for x in I:
    C_i[x] = choice(R)
    

In [4]:
datos_carreras["dia"] = datos_carreras.Fecha.apply(lambda x: time.strptime(x, "%d-%m-%Y").tm_yday)
dias = datos_carreras["dia"]
regiones = datos_carreras["Region"]
F_lt = defaultdict(dict)
for a in range(len(dias)):
    r = regiones[a]
    d = dias[a]
    lugares = L_r[r]
    for l in lugares:
        if F_lt[l].get(d) is None:
            F_lt[l][d] = 0
            break

In [5]:

for i in T:
    for l in L:
        if F_lt[l].get(i) != 1:
            F_lt[l][i] = int(randint(0, 100) < 90)

In [6]:
Y_l = defaultdict(int)
for l in L:
    Y_l[l] = randint(100000, 500000)

In [7]:
D_rs = defaultdict(dict)
for i in range(0,len(R)):
    for x in range(len(R) - i):
        D_rs[R[x]][R[x+i]] = i
        D_rs[R[x+i]][R[x]] = i

In [8]:

P_jt=defaultdict(int)
P_jr=defaultdict(str)

Mu_l = defaultdict(int)
for i in L:
    Mu_l[i] = randint(200, 300)
datos_carreras["dia"] = datos_carreras.Fecha.apply(lambda x: time.strptime(x, "%d-%m-%Y").tm_yday)
for i in range(len(P)):
    r = datos_carreras["Region"].tolist()[i]
    t = datos_carreras["dia"].tolist()[i]
    P_jt[P[i]] = t
    P_jr[P[i]] = r
    l = choice(L_r[r])
    F_lt[l][t] = 1

    

In [9]:
H = 12000
K_r = defaultdict(int)
for x in R:
    K_r[x] = 100
Ohm = 1000
Libra = 1000



In [10]:
U_it = defaultdict(dict)
for i in I:
    for t in T:
        U_it[i][t] = 0
    #for t in datos_carreras["dia"].tolist():
    #    U_it[i][t] = int(randint(1, 100)  >  92)


In [11]:
with open("parametros/F_lt.csv", "w") as file:
    line=";"
    for t in T:
        line += str(t)+";"
    file.write(line + "\n")
    for l, aux in F_lt.items():
        line = str(l) + ";"
        for t, val in aux.items():
            line += str(val) + ";"
        file.write(line + "\n")

with open("parametros/D_rs.csv", "w") as file:
    line=";"
    for r in R:
        line += str(r)+";"
    file.write(line + "\n")
    for r, aux in D_rs.items():
        line = str(r) + ";"
        for s, val in aux.items():
            line += str(val) + ";"
        file.write(line + "\n")


with open("parametros/U_it.csv", "w") as file:
    line=";"
    for t in T:
        line += str(t)+";"
    file.write(line + "\n")
    for i, aux in U_it.items():
        line = str(i) + ";"
        for s, val in aux.items():
            line += str(val) + ";"
        file.write(line + "\n")


        
        
##################### largo 1
with open("parametros/Mu_l.csv", "w") as file:
    file.write("lugar;capacidad\n")
    for x, y in Mu_l.items():
        file.write("{};{}\n".format(x,y))

with open("parametros/Y_l.csv", "w") as file:
    file.write("lugar;precio\n")
    for x, y in Y_l.items():
        file.write("{};{}\n".format(x,y))

with open("parametros/C_i.csv", "w") as file:
    file.write("corredor;region\n")
    for x, y in C_i.items():
        file.write("{};{}\n".format(x,y))
def wri(arch, var):
    arch.write("{};{}\n".format(var, eval(var)))

################### constantes
with open("parametros/constantes.csv", "w") as file:
    wri(file, "A")
    wri(file, "B")
    wri(file, "H")
    wri(file, "Libra")
    wri(file, "Ohm")

In [12]:
### Conjuntos ###
"""
I: corredores
T: dias del año
R: regiones
X: categorias
P: carreras ya agendadas
Q: carreras a agendar
L: lugares disponibles
E: edades

"""
### parametros ###
"""
Constantes
A: presupuesto
B: numero minimo de carreras a agendar
H: costo incripcion carrera
Libra: multa por agendar en xla misma fecha q exista otra carrera
Ohm: multa por distancia entre region x y region y

conjuntos 1D:
C_i: region corredor i
Y_l: precio de arrendar lugar l
...



"""
""

''

In [13]:
from gurobipy import *


In [14]:
model = Model("Planificacion Carreras MTB2")

## Variables

*	𝑉_𝑖𝑗 = 1 ssi el corredor i ∈ I asiste a la carrera j ∈ Q. 0 en otro caso
*	𝑍_𝑗𝑡 = 1 ssi se agenda la carrera j ∈ Q en la fecha t ∈ T. 0 en otro caso
*	𝑀_𝑗𝑟 = 1 ssi la carrera j ∈ Q se corre en la región r ∈ R. 0 en otro caso 
*	𝑆_𝑗𝑙 = 1 ssi la carrera j ∈ Q se corre en el lugar l ∈ L. 0 en otro caso


In [15]:

def add_Var(c1, c2, var):
    aux = eval(var + "_" + c1.lower() + c2.lower())
    if c1 != "j":
        conjunto1 = eval(c1.upper())
    else:
        conjunto1 = Q
    if c2 != "j":
        conjunto2 = eval(c2.upper())
    else:
        conjunto2 = Q
    for x in conjunto1:
        for y in conjunto2:
            aux[x,y] = model.addVar(vtype=GRB.BINARY, name="{}_{}{}".format(var,c1,c2))

V_ij = {}
Z_jt = {}
M_jr = {}
S_jl = {}
Y_jr = {}



add_Var("i", "j", "V")
add_Var("j", "t", "Z")
add_Var("j", "r", "M")
add_Var("j", "l", "S")


# Llama a update para agregar las variables al modelo
model.update()

In [16]:
# Construcciones de variables
# R1
for j in Q:
    for r in R:
        model.addConstr(
            M_jr[j,r] == quicksum([ S_jl[(j,l)] for l in L_r[r] ]),
            "R1MS_{}_{}".format(j,r)
        )

In [17]:
# R2
for j in Q:
    for i in I:
        model.addConstr(
            quicksum([S_jl[j,l] for l in L ]) >= V_ij[i,j],
            "R2SV_{}{}".format(j,i)
        )


In [18]:
# R3
for j in Q:
    model.addConstr(
        quicksum([S_jl[j,l] for l in L]) == quicksum([Z_jt[j,t] for t in T]),
        "R3j_{}".format(j)
    )


In [19]:
# R4

for i in I:
    for j in Q:
        for t in T:
            model.addConstr(
                V_ij[i,j] <= 1 - U_it[i][t],
                "R4VZU_{}_{}_{}".format(i,j,t)
            )

In [20]:
# R3'

for j in Q:
    model.addConstr(
        quicksum([ S_jl[j,l] for l in L])<= 1,
        "R32_{}".format(j)
    )

In [21]:
# R5

for j in Q:
    model.addConstr(
        quicksum([ V_ij[i,j] for i in I]) <= quicksum([Mu_l[l] * S_jl[j,l] for l in L] ),
                "R5_{}".format(j)
    )

In [22]:
# R6

for t in T:
    for j in Q:
        model.addConstr(
            Z_jt[j,t] <= quicksum([F_lt[l][t] for l in L ]),
            "R6_{}_{}".format(j,t)
        )



In [23]:
# R7


for j in Q:
    for l in L:
        model.addConstr(
            S_jl[j,l]<=quicksum([F_lt[l][t] *  Z_jt[j,t] for t in T ]),
            "R7_{}_{}".format(t,j,l)
        )

In [24]:
# R8
for r in R:
    model.addConstr(
        quicksum([int(P_jr[j] == r) for j in P]) + quicksum([M_jr[j,r]  for j in Q]) <= K_r[r],
        "R8_{}".format(r)
    )
            

In [25]:
# R9

for j in Q:
    for q in Q:
        for t in T[:-1]:
            if j != q:
                try:
                    model.addConstr(
                        Z_jt[j,t] + Z_jt[q, t+1] <= 1,
                        "R9_{}_{}_{}".format(j,q,t)
                    )
                except:
                    pass
                try:
                    model.addConstr(
                        Z_jt[j,t] + Z_jt[q, t+6] <= 1,
                        "R9_{}_{}_{}".format(j,q,t)
                    )
                except:
                    pass
                try:
                    model.addConstr(
                        Z_jt[j,t] + Z_jt[q, t+7] <= 1,
                        "R9_{}_{}_{}".format(j,q,t)
                    )
                except:
                    pass

In [26]:
# R10
model.addConstr(
    quicksum([quicksum([S_jl[j,l] * Y_l[l] for l in L]) for j in Q]) <= A,
    "R12"
)

<gurobi.Constr *Awaiting Model Update*>

In [27]:
# R11

model.addConstr(
    quicksum([quicksum([S_jl[j,l] for l in L]) for j in Q]) >= B,
    "R131"
)
model.addConstr(
    quicksum([quicksum([Z_jt[j,t] for t in T]) for j in Q]) >= B,
    "R132"
)

<gurobi.Constr *Awaiting Model Update*>

In [28]:
# R12

for j in Q:
    for q in Q:
        for t in T:
            if j != q:
                model.addConstr(
                    Z_jt[j,t] + Z_jt[q, t] <= 1,
                    "R13_{}_{}_{}".format(j,q,t)
                )


In [29]:
# FO

p1 = quicksum([quicksum([V_ij[i,j] * H for i in I ]) for j in Q])


In [30]:
p2 = quicksum([Y_l[l] * quicksum([S_jl[j,l] for j in Q]) for l in L ])


In [31]:
for x, y in P_jt.items():
    if i not in T:
        for w in T:
            if w <= y + 7:
                P_jt[x] = w
P_jt["xco la ronciere-18"] = 167
p3 = quicksum([ quicksum( [ Z_jt[j,P_jt[i]] * Libra for i in P ] ) for j in Q ])


In [32]:
p4 = quicksum( [ quicksum([ quicksum([Ohm * D_rs[C_i[i]][r] * V_ij[i,j] * M_jr[j,r] for i in I ]) for r in R ]) for j in Q])


In [33]:
model.update()
model.setObjective( p1 -p2 -p3 -p4, GRB.MAXIMIZE)

model.optimize()

Optimize a model with 587760 rows, 6630 columns and 749120 nonzeros
Model has 31920 quadratic objective terms
Variable types: 0 continuous, 6630 integer (6630 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+03, 5e+05]
  QObjective range [2e+03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+06]
Found heuristic solution: objective -1761458.000
Presolve removed 580226 rows and 10 columns
Presolve time: 1.53s
Presolved: 39454 rows, 38540 columns, 152690 nonzeros
Variable types: 0 continuous, 38540 integer (38540 binary)

Root relaxation: objective 3.259577e+07, 5563 iterations, 0.89 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.2596e+07    0 5372 -1761458.0 3.2596e+07  1950%     -    3s
H    0     0                    2.567863e+07 3.2596e+07  26.9%     -    3s
H    0     0                    2.6035

     0     0 3.1804e+07    0 5405 2.9288e+07 3.1804e+07  8.59%     -   57s
     0     0 3.1801e+07    0 5405 2.9288e+07 3.1801e+07  8.58%     -   57s
     0     0 3.1799e+07    0 5405 2.9288e+07 3.1799e+07  8.57%     -   57s
     0     0 3.1797e+07    0 5405 2.9288e+07 3.1797e+07  8.57%     -   57s
     0     0 3.1794e+07    0 5405 2.9288e+07 3.1794e+07  8.56%     -   57s
     0     0 3.1792e+07    0 5405 2.9288e+07 3.1792e+07  8.55%     -   57s
     0     0 3.1789e+07    0 5405 2.9288e+07 3.1789e+07  8.54%     -   57s
     0     0 3.1787e+07    0 5405 2.9288e+07 3.1787e+07  8.53%     -   58s
     0     0 3.1784e+07    0 5405 2.9288e+07 3.1784e+07  8.52%     -   58s
     0     0 3.1782e+07    0 5399 2.9288e+07 3.1782e+07  8.52%     -   58s
     0     0 3.1779e+07    0 5399 2.9288e+07 3.1779e+07  8.51%     -   58s
     0     0 3.1777e+07    0 5399 2.9288e+07 3.1777e+07  8.50%     -   58s
     0     0 3.1775e+07    0 5399 2.9288e+07 3.1775e+07  8.49%     -   59s
     0     0 3.1773e+07  

     0     0 3.1575e+07    0 5271 2.9299e+07 3.1575e+07  7.77%     -   90s
     0     0 3.1572e+07    0 5274 2.9299e+07 3.1572e+07  7.76%     -   91s
     0     0 3.1568e+07    0 5274 2.9299e+07 3.1568e+07  7.74%     -   91s
     0     0 3.1565e+07    0 5269 2.9299e+07 3.1565e+07  7.73%     -   91s
     0     0 3.1562e+07    0 5274 2.9299e+07 3.1562e+07  7.72%     -   92s
     0     0 3.1560e+07    0 5277 2.9299e+07 3.1560e+07  7.72%     -   92s
     0     0 3.1560e+07    0 5278 2.9299e+07 3.1560e+07  7.71%     -   92s
     0     0 3.1525e+07    0 5295 2.9299e+07 3.1525e+07  7.60%     -   96s
     0     0 3.1521e+07    0 5299 2.9299e+07 3.1521e+07  7.58%     -  102s
     0     0 3.1517e+07    0 5285 2.9299e+07 3.1517e+07  7.57%     -  103s
     0     0 3.1513e+07    0 5279 2.9299e+07 3.1513e+07  7.56%     -  103s
     0     0 3.1509e+07    0 5284 2.9299e+07 3.1509e+07  7.54%     -  103s
     0     0 3.1506e+07    0 5281 2.9299e+07 3.1506e+07  7.53%     -  104s
     0     0 3.1502e+07  

  1033   773 3.0178e+07   13 5566 2.9363e+07 3.0681e+07  4.49%   471  490s
  1035   774 2.9910e+07   14 5372 2.9363e+07 3.0681e+07  4.49%   470  498s
  1036   775 2.9536e+07   26 5196 2.9363e+07 3.0681e+07  4.49%   470  503s
  1037   776 2.9416e+07   38 5444 2.9363e+07 3.0681e+07  4.49%   469  506s
  1039   777 2.9577e+07   27 6110 2.9363e+07 3.0681e+07  4.49%   468  514s
  1040   778 3.0121e+07   10 6182 2.9363e+07 3.0681e+07  4.49%   468  515s
  1046   782 2.9857e+07   14 6072 2.9363e+07 3.0681e+07  4.49%   465  520s
  1054   787 3.0192e+07   11 6145 2.9363e+07 3.0681e+07  4.49%   461  525s
  1062   792 2.9704e+07   20 6144 2.9363e+07 3.0681e+07  4.49%   458  530s
  1067   796 2.9398e+07   34 6051 2.9363e+07 3.0681e+07  4.49%   456  535s
  1076   802 3.0157e+07   14 6048 2.9363e+07 3.0681e+07  4.49%   452  540s
  1086   808 3.0167e+07   10 6002 2.9363e+07 3.0681e+07  4.49%   448  545s
  1095   814 2.9672e+07   18 6002 2.9363e+07 3.0681e+07  4.49%   444  551s
  1100   818 2.9650e+07  

  1699   930     cutoff   26      2.9363e+07 2.9565e+07  0.69%   608 1975s
  1746   924     cutoff   29      2.9363e+07 2.9564e+07  0.68%   603 1981s
  1771   925 infeasible   29      2.9363e+07 2.9564e+07  0.68%   600 2001s
  1813   925 2.9429e+07   23 4547 2.9363e+07 2.9549e+07  0.63%   597 2009s
  1839   922     cutoff   28      2.9363e+07 2.9540e+07  0.60%   597 2014s
  1869   926 2.9404e+07   24 2911 2.9363e+07 2.9540e+07  0.60%   597 2019s
  1900   925 2.9383e+07   25 3240 2.9363e+07 2.9540e+07  0.60%   596 2048s
  1920   916     cutoff   27      2.9363e+07 2.9530e+07  0.57%   595 2051s
  1993   898     cutoff   25      2.9363e+07 2.9520e+07  0.53%   587 2058s
  2029   904     cutoff   25      2.9363e+07 2.9513e+07  0.51%   582 2062s
  2069   894 2.9409e+07   23 6015 2.9363e+07 2.9508e+07  0.49%   579 2079s
  2104   890     cutoff   25      2.9363e+07 2.9500e+07  0.47%   576 2083s
  2147   873 2.9463e+07   21 4294 2.9363e+07 2.9497e+07  0.45%   573 2111s
  2180   871 2.9373e+07  

In [34]:
def writeR(var, c1, c2):
    with open("resultados/"+var+".csv", "w") as file:
        file.write("{};{};valor\n".format(c1,c2))
        for x, y in eval(var).items():
            a,b = x
            file.write("{};{};{}\n".format(a,b,int(y.X)))
writeR("V_ij", "corredor", "carrera")
writeR("Z_jt", "carrera", "fecha")
writeR("M_jr", "carrera", "carrera")
writeR("S_jl", "carrera", "lugar")

In [35]:
u = 0
num = set()
for i in I:
    a = 0
    for c in Q:
        if V_ij[i, c].X > 0.1:
            a += 1
    if a > 0:
        print(i)
        u+=1
    num.add(a)
print(u)
num

Benjamin andres vasquez figueroa
Daniel esteban hernandez ardura
Alberto ignacio campos mardones
Francisco andres escobar chavez
Daigorot munoz cortez
Tomas sanhueza vasquez
Augusto cortes labrin
Diego francisco ayala romero
Sebastian alberto miranda marquez
Sebastian alexis maldonado maldonado
Matias eduardo balboa jara
Rodrigo andres coronel fuentealba
Francisco javier beroiza castillo
Daryl edgardo gonzalez hernandez
Gonzalo eduardo gracia loch
Luis campos
Cristian hernan quevedo quevedo
Fernando andres madariaga alarcon
Mauricio alejandro joel cid jara
Alex manuel marquez fuentes
Michel paillacar arcos
Cristian catalan toledo
Yanco campos
Michael alejandro bianchi gajardo
Gonzalo alejandro acuna galvez
Gerardo andres perez lagos
Cristian eduardo vilugron aravena
David alejandro rozas cassanelli
Sebastian ignacio castillo castro
Jorge luis benitez
Edgardo antonio salazar quinteros
Andres patricio navarrete corsini
Maximilian mendez fuentes
Mario benedicto diaz escobar
Pol hector que

Campos kiliman ezequiel
Suarez bartolini daniel alberto
Rodriguez la fuente juan ernesto
381


{0, 1, 2, 3, 4, 5, 6, 7, 8, 10}

In [76]:
print(model.getAttr("SolCount"))
lista = []
for i in range(model.getAttr("SolCount")):
    model.setParam("SolutionNumber", i)
    lista.append(model.PoolObjVal)
lista

10
Changed value of parameter SolutionNumber to 0
   Prev: 9  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 1
   Prev: 0  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 2
   Prev: 1  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 3
   Prev: 2  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 4
   Prev: 3  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 5
   Prev: 4  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 6
   Prev: 5  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 7
   Prev: 6  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 8
   Prev: 7  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter SolutionNumber to 9
   Prev: 8  Min: 0  Max: 2000000000  Default: 0


[32043390.0,
 32026283.803354643,
 32160914.0,
 32143808.0,
 32244226.0,
 32361750.0,
 32496380.0,
 32210013.999999996,
 32479274.0,
 32596798.0]

In [41]:
relajada = model.relax()

In [44]:
relajada.write("solucion_relajada.rew")

In [45]:
model.write("modelo.rew")

In [70]:
model.getParamInfo("ObjNumber")

('ObjNumber', int, 0, 0, 2000000000, 0)

In [82]:
model.getObjective()

<gurobi.QuadExpr: 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0 V_ij + 12000.0